# Alphabet Soup Charity

## Preprocessing

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
# import io # for Google Colab

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("./Resources/charity_data.csv") # Local Jupyter Notebook
# application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv'])) # Google Colab
application_df.head(3)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(['EIN','NAME'], axis=1)
df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [5]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_value_counts = df['APPLICATION_TYPE'].value_counts()
app_type_value_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for x in range(len(app_type_value_counts)):
    if app_type_value_counts.values[x] < 500:
        application_types_to_replace.append(app_type_value_counts.index[x])
        
# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_val_counts = df['CLASSIFICATION'].value_counts()
classification_val_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] > 1:
        print(f'{classification_val_counts.index[x]}\t{classification_val_counts.values[x]}')

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] < 1500:
        classifications_to_replace.append(classification_val_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df)
print(df_dummies.columns)
df_dummies

In [ ]:
# Split our preprocessed data into our features and target arrays
y_label = df_dummies['IS_SUCCESSFUL']
X_dummies = df_dummies.drop(['IS_SUCCESSFUL'],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

In [ ]:
X_train.shape

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# from tensorflow.keras.layers import LeakyReLU, Dense
# leaky_relu = LeakyReLU(alpha=0.01)

from tensorflow.keras.initializers import LecunNormal # for use with selu activation function

# First hidden layer
nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=leaky_relu)
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu", kernel_initializer=LecunNormal())
)

# Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=leaky_relu))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="selu", kernel_initializer=LecunNormal()))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
nn.save('AlphabetSoupCharity.h5')
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")